### System rekomendujący na bazie Netflix Prize award

1. Zacznijmy od obejrzenia wykładu [rozdział 9](http://www.mmds.org/)
2. proszę ściągnac bazę netflixa o której była mowa w/w wykładzie [z kaggle](https://www.kaggle.com/netflix-inc/netflix-prize-data) (2GB) 

Zaczniemy od wczytania danych, ponizszy listing wczytuje dane z jednego pliku i robi z nich trójkę (user, product, rating). Wykorzystamy do tego predefiniowany obiekt Rating z mlib.recommendation (prosze zwrócic uwagę że w tej konwencji nasz film będzie produktem)

#### Zadanie 1:
zmodyfikuj poniższy listing tak aby wczytywać wsystkie pięc plików

In [1]:
import pyspark
from pyspark.mllib.recommendation import Rating
from pyspark import SparkContext

# tutaj zaincjalizujemy klaster (na jednym komputerze) sparka
# zmodyfikujemy nieco std ustawienia maszyny java zwiększając domyslne (bardzo małe) limity pamieci
# local[*] oznacza że użyjemy wsystkich rdzeni - jeśli zabrankie nam ramu możemy zmniejszyć ilość tą ilość
# polecam spoglądać do konsoli na http://localhost:4040/ aby monitorować zużycie zasobów

conf = pyspark.SparkConf().setAppName("recommendation")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '20G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)

#files = ['./RS/combined_data_1.txt',
        #'./RS/combined_data_2.txt',
        #'./RS/combined_data_3.txt',
        #'./RS/combined_data_4.txt'] # Ufam, że użytkownik wprowadził nazwy plików, które istnieją...
files = [ './RS/combined_data_1.txt' ]

ratings = []
for single_name in files:
    f = open(single_name)
    for i, line in enumerate(f):
        line = line.strip()
        if line.endswith(':'):
            movie_id = int(line[:-1])        
        else:
            user_id, rating, _ = line.split(',')
            r = Rating(int(user_id), int(movie_id), int(rating))
            ratings.append(r)
    f.close()
    print("File '" + single_name + "' loaded!")

print('Finished! Processed lines: ' + str(len(ratings)))

File './RS/combined_data_1.txt' loaded!
Finished! Processed lines: 24053764


In [2]:
#tutaj wczytamy identyfikatory filmów i ich tytuły

f = open('./RS/movie_titles.csv', encoding = "ISO-8859-1")
g = [l.strip().split(',') for l in f.readlines()]
id2title = {int(a[0]):','.join(a[2:]) for a in g}
f.close()
print('Finished!')

Finished!


In [3]:
id2title
print('Finished!')

Finished!


In [3]:
import time

# utworzmy z naszej listy ratingow zasob rdd (rozproszenie)
start = time.time()
ratings_rdd = sc.parallelize(ratings)
end = time.time()
print('Finished! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

Finished! Elapsed time: 31.34 seconds.


#### Zadanie 2
Przefiltruj ratings_rdd aby wziac pod uwage filmy ktory maja co najmniej 50 ocen
tip: zrób napierw liste par (movie_id, Rating) i pogrupuj ją przy użyciu GroupByKey a nstępnie przefiltruj

In [8]:
ratings_pairs = ratings_rdd.map(lambda r: (r.product, r))
rg = ratings_pairs.groupByKey() # ???

# ???
ratings_filtered = rg.filter(lambda r: len(r[1]) >= 50).collect() # ???


# zasob rdd na przefiltrowanym obiekcie
ratings_filtered = sc.parallelize(ratings_filtered)

PythonRDD[1] at RDD at PythonRDD.scala:53


In [10]:
# TROCHĘ INNYM SPOSOBEM - na około (zmiana na DataFrame po to by korzystać z SQL Queries)
import pyspark.sql.functions as ps_functions
from pyspark.sql import SQLContext
from pyspark.sql import Window
import time

start = time.time()
data = SQLContext(sc)
print('Stage 1 - finished!')

all_ratings = data.createDataFrame(ratings_rdd)
print('Stage 2 - finished!')

fragment = Window.partitionBy('product')
print('Stage 3 - finished!')

query = all_ratings.select('user', 'product', 'rating', ps_functions.count('product').over(fragment).alias("ratings_count"))
print('Stage 4 - finished!')

marker = query.filter("ratings_count >= 50").select('user','product','rating')
print('Stage 5 - finished!')

ratings = sc.parallelize(marker.collect())
print('Stage 6 - finished!')

end = time.time()
print('Finished! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')
# TROCHĘ INNYM SPOSOBEM - na około (zmiana na DataFrame po to by korzystać z SQL Queries)

Stage 1 - finished!
Stage 2 - finished!
Stage 3 - finished!
Stage 4 - finished!
Stage 5 - finished!
Stage 6 - finished!
Finished! Elapsed time: 203.6 seconds.


Dokonamy teraz faktoryzacji macierzy, nasej utility Matrix (zasób RDD ratings_filtered jest własnie taką macierzą) przy użyciu aproksymacji algorytmu spadku gradientowego
![alt](https://edersoncorbari.github.io/assets/images/blog/als-matrix-rec-calc.png)

In [13]:
from pyspark.mllib.recommendation import ALS
import time

rank = 10
numIterations = 10

start = time.time()
model = ALS.train(ratings, rank, numIterations)
end = time.time()
print('Finished! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

Finished! Elapsed time: 79.81 seconds.


Zrzucilismy uzytkowników i filmy na nowy wymiar o wielkości 10. Możemy wprost poprosić o te macierze

In [14]:
users  = model.userFeatures()
movies = model.productFeatures()

Gęsta ale 'wąska' macierz movies zmieści się nam już bezproblemowo w RAM, a zatem zróbmy z niej po prostu macierz numpy

In [16]:
import numpy as np

movies_mtx = np.array(movies.map(lambda rv:rv[1]).collect())
movie2row_number = movies.map(lambda rv:rv[0]).collect() #movie id to row number

Zróbmy prosty ekesperyment, zobaczmy czy jakie filmy są 'podobne' do 'Władcy pierścieni'

In [19]:
id2title[1757] # 'The Lord of the Rings'

lotr_idx = movie2row_number.index(1757)
lotr_vector = movies_mtx[lotr_idx]
print('Tytuł: ' + id2title[1757])
print(lotr_vector) #to jest wladca pierscienie rzutowany na latent space

Tytuł: The Lord of the Rings
[-0.4527832  -0.08132     0.23505069 -0.54495251  0.38770333  0.32258755
 -0.23984745  0.03348436  0.52501208  0.24838813]


In [23]:
# obliczmy macierz odleglosci 
from scipy.spatial import distance

ds = distance.cdist([lotr_vector], movies_mtx, 'cosine')[0]
dist = ds.argsort()[:10] # 10 - liczba filmów

print('Odległości: ' + str(dist))

for i in dist:
    print('Tytuł filmu o ID ' + str(i) + ': ' + id2title[movie2row_number[i]])

Odległości: [1561 4481 2720 3569 2072 4108 2782 1176  313 2449]
Tytuł filmu o ID 1561: The Lord of the Rings
Tytuł filmu o ID 4481: Escaflowne: The Movie
Tytuł filmu o ID 2720: Star Trek: The Motion Picture
Tytuł filmu o ID 3569: The Lawnmower Man
Tytuł filmu o ID 2072: The Clan of the Cave Bear
Tytuł filmu o ID 4108: Burn Up W
Tytuł filmu o ID 2782: The Return of the King
Tytuł filmu o ID 1176: Cherry 2000
Tytuł filmu o ID 313: Journey to the Center of the Earth
Tytuł filmu o ID 2449: Logan's Run


In [28]:
for i in dist:
    print('Wartość odległości: ' + str(ds[i]) + " dotyczy filmu: " + id2title[movie2row_number[i]])

# print(str(ds[2018]) + ': ' + id2title[movie2row_number[2018]])
# print(str(ds[928]) + ': ' + id2title[movie2row_number[928]])

Wartość odległości: 0.0 dotyczy filmu: The Lord of the Rings
Wartość odległości: 0.07878517057753731 dotyczy filmu: Escaflowne: The Movie
Wartość odległości: 0.08781767506452776 dotyczy filmu: Star Trek: The Motion Picture
Wartość odległości: 0.08937511326538616 dotyczy filmu: The Lawnmower Man
Wartość odległości: 0.09714696540534362 dotyczy filmu: The Clan of the Cave Bear
Wartość odległości: 0.09870806179188363 dotyczy filmu: Burn Up W
Wartość odległości: 0.09989895667984683 dotyczy filmu: The Return of the King
Wartość odległości: 0.1067716968402107 dotyczy filmu: Cherry 2000
Wartość odległości: 0.10821727445284046 dotyczy filmu: Journey to the Center of the Earth
Wartość odległości: 0.10967764047823081 dotyczy filmu: Logan's Run


Odległosć 0.0 to oczywiscie ten sam obiekt a zatem interesuje nas drugi w kolejności wpis. I cóż za niespodzianka? ludzie którzy ocenili wysoko Władce pierścieni ocenili również wysokos Powrót Króla

Na ile cech powinniśmy faktoryzować Utility Matrix? 

#### Zadanie 3
Dobierz paramter rank (ilość cech) dla obiektu ALS dzieląc zbiór ratings_filtered na 80% vs 20% (treningowy i testowy, tak jak na wykłdzie). Następnie wykorzystać metodę ALS.recommendProductsForUsers aby polecić filmy i zweryfikuj ile filmów udało Ci sie prawidłowo polecić ze zbioru testowego. Odpowiednio modyfikuj parametr rank aby otrzymać względnie wysoki wynik rekomendacji jednocześnie utrymując w miare wąskie macierze

In [ ]:
import pyspark
from pyspark.mllib.recommendation import Rating
from pyspark import SparkContext
import time

conf = pyspark.SparkConf().setAppName("recommendation")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '20G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)

#files = ['./RS/combined_data_1.txt',
        #'./RS/combined_data_2.txt',
        #'./RS/combined_data_3.txt',
        #'./RS/combined_data_4.txt'] # Ufam, że użytkownik wprowadził nazwy plików, które istnieją...
files = [ './RS/combined_data_1.txt' ]

ratings = []
for single_name in files:
    f = open(single_name)
    for i, line in enumerate(f):
        line = line.strip()
        if line.endswith(':'):
            movie_id = int(line[:-1])        
        else:
            user_id, rating, _ = line.split(',')
            r = Rating(int(user_id), int(movie_id), int(rating))
            ratings.append(r)
    f.close()
    print("File '" + single_name + "' loaded!")

print('Loading finished! Processed lines: ' + str(len(ratings)))

f = open('./RS/movie_titles.csv', encoding = "ISO-8859-1")
g = [l.strip().split(',') for l in f.readlines()]
id2title = {int(a[0]):','.join(a[2:]) for a in g}
f.close()
print('IDs and titles loaded!!')

start = time.time()
ratings_rdd = sc.parallelize(ratings)
end = time.time()
print('RDD resource created! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

In [5]:
import pyspark.sql.functions as ps_functions
from pyspark.sql import SQLContext
from pyspark.sql import Window
import time

start = time.time()
data = SQLContext(sc)
print('Stage 1 - finished!')

all_ratings = data.createDataFrame(ratings_rdd)
print('Stage 2 - finished!')

fragment = Window.partitionBy('product')
print('Stage 3 - finished!')

query = all_ratings.select('user', 'product', 'rating', ps_functions.count('product').over(fragment).alias("ratings_count"))
print('Stage 4 - finished!')

marker = query.filter("ratings_count >= 50").select('user','product','rating')
print('Stage 5 - finished!')

ratings = sc.parallelize(marker.collect())
print('Stage 6 - finished!')

end = time.time()
print('Finished! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

Stage 1 - finished!
Stage 2 - finished!
Stage 3 - finished!
Stage 4 - finished!
Stage 5 - finished!
Stage 6 - finished!
Finished! Elapsed time: 167.79 seconds.


In [10]:
train, validate, test = ratings.randomSplit([6,2,2], seed=0)

emptyTest = test.map(lambda r: (r[0], r[1]))

emptyValidation = validate.map(lambda r: (r[0], r[1]))

#### Zadanie 4
Zrób program "co obejrzeć dziś wieczorem".
Program powinien pokazaywać 5 tytułow, uzytkownik wskazuje jeden lub kilka z nich i system pokazuje kolejne rekomendacje. Każda kolejna rekomendacja powinna wskazywać coraz lepsze rekomendacje. 

problem można rozwiazać na kilka sposób. Można pogrupować przestrzeń filmów (macierz P.T) i pokazywać z różnych np. centroidy klastrów (które będą reprezentować gatunki filmów (prawdopodobnie) ). 

każde kolejne wybranie filmu będzie wymagało stworzenie nowego sztuznego użytkownika z ratingiem. W każdej iteracji takiego użytkownika należy rzucic na macierz Q (czyli na latentną macierz użytkownika). Przypatrz się dokładnie sposobowi mnożenia na obrazku powyżej i zastanów się jak otrzymać takiego użytkownika w przestrzeni cech?). Przypominam że macierz P.T jest nieodracalna ale można wykorzystać pseudo odwrotnosć.